# Bank Churners
**ML Project 1 - Spring 2022**

Collaborators: Ben DeSollar and Matt McDonnell

Descriptions: The first part should be a description which models/classifiers/regressors achieved the best 
    results with your datasets datasets, and what you did to improve those results along the way.

    - Through research we found that a called RandomForestClassifier worked the best after testing a couple different model including
    Niave Bayes and DecisionTreeClassifier. We improved our results by modifying our feature selection to include as many features as we 
    could to help predict the best values. Things like gender and Total relationship count did not seem as impptant as education level or
    credit limit. By modifying our featuiues during testing we were able to find the best model predictions.



### Step 1
Import the necessary packages and frameworks.

In [193]:
import numpy as np
from mlwpy import *
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import seaborn as sns
from sklearn import (datasets, neighbors,
                     naive_bayes,
                     model_selection as skms,
                     linear_model, dummy,
                     metrics,
                     pipeline,
                     preprocessing as skpre) 
import csv
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier

### Step 2
Read in .csv files and save as DataFrames.

In [194]:
data_train_df = pd.read_csv("cars_class_train.csv", nrows=10000) 
data_test_df = pd.read_csv("cars_class_test.csv", nrows=10000)
data_train_ft = data_train_df.drop('price', axis=1)
data_train_tgt = data_train_df["price"]

### Step 3
Work with only the necessary features. We tried to choose features that were only about the users finances and not cateogies like gender or relationship count.
Yet things like marital status and education level seemed to have an impact on the prediction scores. 

In [195]:
features = ['year',\
            'odometer',
            #'lat',
            #'long',
            'drive',
            'size',
            'type',
            #'posting_date',
            'fuel',
            'condition',
            # 'model',
            'paint_color',
            'cylinders',
            'manufacturer'
            ]

# To see what kind of data we're working with
print(data_train_ft.dtypes)
print(data_train_ft['manufacturer'].unique())
# print(data_train_ft.head(5))

Unnamed: 0        int64
url              object
region           object
region_url       object
year            float64
manufacturer     object
model            object
condition        object
cylinders        object
fuel             object
odometer        float64
title_status     object
transmission     object
VIN              object
drive            object
size             object
type             object
paint_color      object
image_url        object
description      object
state            object
lat             float64
long            float64
posting_date     object
dtype: object
['ram' 'chevrolet' 'toyota' 'honda' 'ford' nan 'kia' 'mazda' 'dodge'
 'mercedes-benz' 'gmc' 'nissan' 'jeep' 'mini' 'bmw' 'audi' 'hyundai'
 'mercury' 'volvo' 'chrysler' 'volkswagen' 'subaru' 'infiniti'
 'mitsubishi' 'cadillac' 'lincoln' 'rover' 'buick' 'alfa-romeo' 'lexus'
 'tesla' 'saturn' 'acura' 'pontiac' 'jaguar' 'fiat' 'porsche' 'ferrari'
 'land rover' 'harley-davidson' 'datsun' 'aston-martin']


### Step 4
Work with a practice estimate (median age). Convert features to numerical values as necessary. 

For now, let's use a simple approach to estimate the age (and consider revisiting this estimate later) when it is missing by using the median from the other samples.

We also replaced any missing data in the dataset.

In [196]:


data_train_ft['year'] = data_train_ft['year'].fillna(0)

data_train_ft['odometer'] = data_train_ft['odometer'].fillna(0)

data_train_ft['lat'] = data_train_ft['lat'].fillna(0)

data_train_ft['long'] = data_train_ft['long'].fillna(0)

data_train_ft['drive'] = data_train_ft['drive'].fillna(0)
data_train_ft['drive'].replace(['4wd','rwd','fwd'],[1, 2, 3],inplace=True)

data_train_ft['size'] = data_train_ft['size'].fillna(0)
data_train_ft['size'].replace(['full-size','compact','mid-size','sub-compact'],[1, 2, 3, 4],inplace=True)

data_train_ft['fuel'] = data_train_ft['size'].fillna(0)
data_train_ft['fuel'].replace(['gas','diesel','other','hybrid','electric'],[1, 2, 3, 4, 5],inplace=True)

data_train_ft['condition'] = data_train_ft['size'].fillna(0)
data_train_ft['condition'].replace(['excellent','fair','like new','good','new','salvage'],[1, 2, 3, 4, 5, 6],inplace=True)

data_train_ft['type'] = data_train_ft['type'].fillna(0)
data_train_ft['type'].replace(['truck','coupe','pickup','SUV','sedan','offroad','hatchback','van',\
 'other','mini-van','wagon','convertible','bus'],[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13],inplace=True)

data_train_ft['title_status'] = data_train_ft['size'].fillna(0)
data_train_ft['title_status'].replace(['clean','rebuilt','salvage','lien','missing','parts only'],[1, 2, 3, 4, 5, 6],inplace=True)


data_train_ft['paint_color'] = data_train_ft['size'].fillna(0)
data_train_ft['paint_color'].replace(['green','white','silver','black','red','yellow','grey','blue'\
 ,'custom','orange','brown','purple'],[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],inplace=True)

data_train_ft['cylinders'] = data_train_ft['size'].fillna(0)
data_train_ft['cylinders'].replace(['6 cylinders','4 cylinders','8 cylinders','5 cylinders',\
 '10 cylinders','3 cylinders','12 cylinders','other'],[1, 2, 3, 4, 5, 6, 7, 8],inplace=True)


data_train_ft['posting_date'] = pd.to_datetime(data_train_ft['posting_date'], utc=True)
data_train_ft['posting_date'] = pd.to_datetime(data_train_ft['posting_date'])
data_train_ft['posting_date'] = (pd.DatetimeIndex(data_train_ft['posting_date']).day) + (pd.DatetimeIndex(data_train_ft['posting_date']).month*12) + ((2022 - pd.DatetimeIndex(data_train_ft['posting_date']).year)*365)
data_train_ft['posting_date'] = data_train_ft['posting_date'].astype(float)
data_train_ft['posting_date'] = data_train_ft['posting_date'].fillna(0)

#data_train_ft['manufacturer'] = data_train_ft['manufacturer'].fillna(0)
i = 0
for unique_val in data_train_ft['manufacturer'].unique():
    data_train_ft['manufacturer'].replace([unique_val],[i],inplace=True)
    i+=1


data_train_ft = data_train_ft[features]
#print(data_train_ft.dtypes)
#print(data_train_ft.head(5))
for col in data_train_ft:
    print(f'Column ({col}) : {data_train_ft[col].unique()}')

Column (year) : [2011. 1954. 2013. 2014. 2019. 2012. 2008. 2005. 2002. 2006. 1988. 2017.
 2020. 2018. 1997. 2010. 2015. 1966. 2016. 2009. 2007. 2000. 1969. 2001.
 2004. 1965. 2003. 1978. 1998. 1995. 1996. 1980. 1984. 1989. 1974. 1999.
 1976.    0. 1979. 1993. 1961. 1994. 1950. 1992. 1946. 1991. 1975. 1968.
 1932. 1963. 1983. 1985. 1941. 1926. 1970. 1972. 1981. 1931. 1940. 1990.
 1958. 1977. 1967. 1959. 1973. 1936. 1987. 1964. 1930. 1960. 1986. 1949.
 2021. 1915. 1982. 1951. 1955. 1957. 1956. 1935. 1948. 1927. 1953. 1962.
 1971. 1938. 1925.]
Column (odometer) : [155815.  72953.      0. ... 137248.  80850.  62960.]
Column (drive) : [1 2 3 0]
Column (size) : [0 1 2 3 4]
Column (type) : [ 1  2  3  4  5  6  7  8  9 10 11 12 13]
Column (fuel) : [0 1 2 3 4]
Column (condition) : [0 1 2 3 4]
Column (paint_color) : [0 1 2 3 4]
Column (cylinders) : [0 1 2 3 4]
Column (manufacturer) : [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 3

Here is an updated table after sorting through these features.

In [197]:
print("data_train_df:")
display(data_train_df.head(10))
print("data_train_df.info():")
display(data_train_df.info())

# Analysis of the features looks like Education level, Credit Limit, and Income Level may be useful. 
# Features will need to be transformed to convert all into numbers to be analyzed.

data_train_df:


,Unnamed: 0,url,region,region_url,price,year,manufacturer,model,condition,cylinders,...,drive,size,type,paint_color,image_url,description,state,lat,long,posting_date
0,154496,https://desmoines.craigslist.org/ctd/d/des-moi...,des moines,https://desmoines.craigslist.org,8995,"2,011.0000",ram,dakota,excellent,6 cylinders,...,4wd,NaN,truck,NaN,https://images.craigslist.org/00t0t_cojzh6OLdk...,NaN,ia,41.6002,-93.5701,2020-11-18T11:04:29-0600
1,340882,https://pittsburgh.craigslist.org/cto/d/greens...,pittsburgh,https://pittsburgh.craigslist.org,4500,"1,954.0000",chevrolet,210,fair,6 cylinders,...,rwd,full-size,coupe,green,https://images.craigslist.org/00101_7LqQvihSgb...,1954 Chevy 210 two door sedan. The car is all ...,pa,40.3074,-79.5424,2020-11-30T10:21:12-0500
2,57036,https://slo.craigslist.org/ctd/d/san-luis-obis...,san luis obispo,https://slo.craigslist.org,16495,"2,013.0000",toyota,tacoma regular cab,excellent,4 cylinders,...,rwd,compact,truck,white,https://images.craigslist.org/00E0E_3doy3u82Gh...,2013 TOYOTA TACOMA REGULAR CAB 2.7L 4 CYLINDER...,ca,35.2468,-120.6676,2020-11-13T21:09:40-0800
3,244897,https://raleigh.craigslist.org/ctd/d/raleigh-2...,raleigh / durham / CH,https://raleigh.craigslist.org,14900,"2,014.0000",honda,accord coupe,like new,6 cylinders,...,fwd,compact,coupe,silver,https://images.craigslist.org/00Z0Z_1IqAlFGmg6...,We are selling a 2014 Honda Accord EX-L Coupe ...,nc,35.7489,-78.6434,2020-11-25T17:02:03-0500
4,206753,https://porthuron.craigslist.org/ctd/d/point-e...,port huron,https://porthuron.craigslist.org,35990,"2,019.0000",ford,f150 supercrew cab xlt,good,6 cylinders,...,NaN,NaN,pickup,black,https://images.craigslist.org/00T0T_cLmS54epjE...,Carvana is the safer way to buy a car During t...,mi,43.0172,-82.4510,2020-11-22T11:09:41-0500
5,346078,https://york.craigslist.org/ctd/d/hanover-2012...,york,https://york.craigslist.org,12900,"2,012.0000",toyota,rav4 limited,NaN,4 cylinders,...,4wd,NaN,SUV,black,https://images.craigslist.org/00r0r_jGSql9WYuT...,2012 Toyota RAV4 Limited STOCK #: S01382 Bad...,pa,39.8264,-76.9991,2020-11-10T17:10:42-0500
6,306950,https://toledo.craigslist.org/ctd/d/bowling-gr...,toledo,https://toledo.craigslist.org,6999,"2,014.0000",chevrolet,cruze,good,4 cylinders,...,fwd,NaN,sedan,red,https://images.craigslist.org/00O0O_4efNgjjYJe...,"2014 Chevrolet Cruze 2LT Auto - $6,999 Year: ...",oh,41.3900,-83.6511,2020-11-27T13:22:06-0500
7,336045,https://lancaster.craigslist.org/ctd/d/south-a...,lancaster,https://lancaster.craigslist.org,34990,"2,011.0000",NaN,International 4300,good,6 cylinders,...,rwd,full-size,truck,yellow,https://images.craigslist.org/00V0V_bYpb4Cx3wD...,"2011 International 4300 CREW CAB, 11' 7"" STAKE...",pa,40.4688,-74.2817,2020-11-11T02:53:10-0500
8,280296,https://newyork.craigslist.org/brk/cto/d/brook...,new york city,https://newyork.craigslist.org,8975,"2,012.0000",kia,sorento,like new,6 cylinders,...,fwd,full-size,offroad,green,https://images.craigslist.org/00L0L_eUhLpzM6OM...,2012 KIA SORENTO NICE LOOKING ONLY 36K MILES...,ny,40.5829,-73.9679,2020-12-01T17:22:42-0500
9,347282,https://providence.craigslist.org/cto/d/barrin...,rhode island,https://providence.craigslist.org,4000,"2,008.0000",mazda,6,good,4 cylinders,...,fwd,full-size,sedan,grey,https://images.craigslist.org/01515_g5N8kjDaTP...,"2008 mazda6 for sale, have upgraded to a new M...",ri,41.7262,-71.3347,2020-11-21T12:51:01-0500


data_train_df.info():
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 25 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    10000 non-null  int64  
 1   url           10000 non-null  object 
 2   region        10000 non-null  object 
 3   region_url    10000 non-null  object 
 4   price         10000 non-null  int64  
 5   year          9967 non-null   float64
 6   manufacturer  9702 non-null   object 
 7   model         9938 non-null   object 
 8   condition     6780 non-null   object 
 9   cylinders     7402 non-null   object 
 10  fuel          9931 non-null   object 
 11  odometer      9500 non-null   float64
 12  title_status  9954 non-null   object 
 13  transmission  9953 non-null   object 
 14  VIN           6765 non-null   object 
 15  drive         8287 non-null   object 
 16  size          3783 non-null   object 
 17  type          10000 non-null  object 
 18  paint

None

### Step 5
Convert all feature values to floating-point numbers for better accuracy when performing tests later.

In [198]:
data_train_ft = data_train_ft.astype(float) 
#data_test_df = data_test_df[features].astype(float) 
data_train_tgt = data_train_tgt.astype(float)
print(data_train_ft.info())

# makes sure all data is uniform in the floating data type

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   year          10000 non-null  float64
 1   odometer      10000 non-null  float64
 2   drive         10000 non-null  float64
 3   size          10000 non-null  float64
 4   type          10000 non-null  float64
 5   fuel          10000 non-null  float64
 6   condition     10000 non-null  float64
 7   paint_color   10000 non-null  float64
 8   cylinders     10000 non-null  float64
 9   manufacturer  10000 non-null  float64
dtypes: float64(10)
memory usage: 781.4 KB
None


### Step 6
Create a library of different models to train and track the accuracy of later. Like the feature selection, having a wide variety of models to test out helps us ensure that our predictions are as accurate as possible.

In [199]:
models_to_try = {'nb': naive_bayes.GaussianNB()}

# add k-NN models with various values of k to models_to_try
for k in range(1,15,3):
    models_to_try[f'{k}-NN'] = neighbors.KNeighborsClassifier(n_neighbors=k)

In [200]:
scaler = skpre.StandardScaler()
pipelines_to_try = \
    {'GNB0' : naive_bayes.GaussianNB(),
   # 'RandomForestClassifier' : RandomForestClassifier()
    }
for k in range(1,15,2):
    models_to_try[f'DTC-{k}'] =  tree.DecisionTreeClassifier(max_depth=k)

baseline = dummy.DummyClassifier(strategy="uniform")

# scale the data
for model_name in models_to_try:
    pipelines_to_try[f'std_{model_name}_pipe'] = pipeline.make_pipeline(scaler, 
                                                      models_to_try[model_name])


sv_classifiers = {"SVC(Linear)"   : svm.SVC(kernel='linear'),
                  "NuSVC(Linear)" : svm.NuSVC(kernel='linear', nu=.9)} 

### Step 8
Use cross-validation to test the accuracy of the models.

In [201]:
accuracy_scores = {}

for name, model in pipelines_to_try.items():
    #loo = skms.LeaveOneOut()
    scores = skms.cross_val_score(model,
                                  data_train_ft,
                                  data_train_tgt,
                                  #cv=loo,
                                  cv=10,
                                  scoring='accuracy')
    mean_accuracy = scores.mean()
    accuracy_scores[name] = mean_accuracy
    print(f'{name}: {mean_accuracy:.3f}')



/Users/bendesollar/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"


GNB0: 0.023


/Users/bendesollar/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"


std_nb_pipe: 0.014


/Users/bendesollar/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"


...

std_DTC-9_pipe: 0.035


/Users/bendesollar/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"


std_DTC-11_pipe: 0.044


/Users/bendesollar/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"


std_DTC-13_pipe: 0.057


### Step 9
Find the pipeline with the highest accuracy and save it for testing.

In [202]:
best_pipeline_name = max(accuracy_scores,key=accuracy_scores.get)
print(f'\nBest pipeline: {best_pipeline_name} (accuracy = {accuracy_scores[best_pipeline_name]:.3f})')
final_pipeline = pipelines_to_try[best_pipeline_name]

# Best model seems to be RandomForestClassifier (accuracy = 0.954)


Best pipeline: std_1-NN_pipe (accuracy = 0.115)


### Step 10
Fit and test the highest-scoring model against the testing data. Save the results in submission.csv for uploading to Kaggle.

In [203]:
scaler.fit(data_train_ft[features])

train_scaled_ftrs = scaler.transform(data_train_ft[features])

# transform validation features
test_scaled_ftrs = scaler.transform(data_test_df[features])

fit = final_pipeline.fit(train_scaled_ftrs, data_train_tgt)
predictions = fit.predict(test_scaled_ftrs)
predictions = predictions.astype(int)
print(predictions)

def writeSubmission(predictions):
   i=6751
   submissionList = []
   for prediction in predictions:
       submissionList.append([str(i), str(prediction)])
       i+=1
   with open('submission.csv', 'w', newline='') as submission:
       writer = csv.writer(submission)
       writer.writerow(['id', 'Target'])
       for row in submissionList:
           writer.writerow(row)

writeSubmission(predictions)

KeyError: "['type'] not in index"